# Определение тематического класса текстов речей TEDx

Этот ноутбук решает задачу многоклассовной классификации субтитров на русском языке к записям TEDx.

Оригиналный текст задачи доступен на ссылке: https://www.kaggle.com/c/2nd-hw-dl-in-nlp

Код осуществляет решение к задаче, употребляя два алгоритма машинного обучения: TfidfVectorizer и SGDClassifier. Данные, входящие в отобранный пайплайн машинного обучения, предварительно обработали с помощью токенизатора и stemmer от библиотеки *nltk*, которые извлекают корень всех слов, присуствующих в текстах.

После процесса предварительной обработки, строки, содержащие все извлеченные корни, вводят в качестве инпута в пайплайн для их последующих векторизации и классификации. Определение наиболее эффективных параметров векторизатора и классификатора реализуется образом конструктора GridSearchCV, который сравнивает множество разных конкурирующих моделей, дифференцированных по основаниям их гиперпараметров, и сохраняет самую эффективную модель. Та модель дальше употребляется для прогнозирования классов тестовых текстов, не наблюдавшихся алгоритмом в процессе обучения. В заключение сройтся файл с прогнозированными решениями для его отправления в Kaggle.

Итог расчета можно оценить образом алгоритма оценки Kaggle. Полученные оценки, private:0.68292 и public:0.64878, входит в первое и второе места в таблицах лидеров.

# Подключение библиотек

### Основые библиотеки

Теперь настроим все необходимые библиотеки, употреблявшие в ходе последуюшего кода.

Начинаемся, установливая numpy и генератор случайных чисел, для воспроизводимости и сопоставимость результатов расчета

In [1]:
import numpy as np
np.random.seed(42)

Основные библиотеки для работы с базами данных и обработки естетсвенного языка

In [2]:
import pandas as pd # Основая библиотека для загрузки баз данных
import nltk # Специализированная библиотека для обработки естественного языка

In [3]:
# Дополнительные употреблявшие библиотеки 
from pprint import pprint
import time

### Конструкторы моделей машинного обучения:

##### 1) Векторизатор. 

Его употребляние совпадает с употреблянием CountVectorizer и TfidfTransformer,
в соответствии с документацией sklearn: 
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

##### 2) Классификатор.

Конструктор SGDClassifier создает модель классификатора, который использует метод стохастического градиентного спуска для схождения к глобальному мининмуму.

Зависимо от целевой функции, передана как параметр модели во время ее создания, коструктор совпадает с моделью SVM или Логистической регрессии, если передаем ему, соответственно, 'hinge' или 'log' в качестве целевой функции.
Конструктор применяет и дополнительные целевые функции, хотя мы их не рассмотрим.

Полная документация здесь: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [5]:
from sklearn.linear_model import SGDClassifier

##### 3) Пайплайн

Конструктор Pipeline, это комфортный метод соединения конструкторов типа "трансформатора" с оценщиком. Вызов метода .fit() созданного конструктора вызывает последовательно методи .fit_transform() трансформаторов, и .fit() оценщика.

Этот конструктор хорошо адаптируется для пойска сетки гиперпараметров с GridSearchCV.

Ссылки: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [6]:
from sklearn.pipeline import Pipeline 

##### 4) GridSearchCV

GridSearchCV осуществляет пойск сетки оптимальных гиперпараметров среди тех, что конструктор получает в форме словаря доступных возможностей.

Документация конструктора: http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html

In [7]:
from sklearn.model_selection import GridSearchCV

# Загрузка данных задачи

### Премеры для обучения моделей

Текст задачи представляет два файла в формате json, содержащие тексты и тематические классы множества речей TEDx, плюс тексты несколько неклассифицированных речей.

Все файлы найдены здесь: https://www.kaggle.com/c/2nd-hw-dl-in-nlp/data

Распределяем данные на два множества: обучающие данные, и данные для тестирования. Загружаем их в два кадра данных, train и test, образом функций библиотеки pandas.

In [8]:
train = pd.read_json('train.json')
train.head()

,@id,class,content,head
0,1848,2,"Я был неописуемо удивлён, узнав о существовани...",{'videourl': 'http://download.ted.com/talks/Ne...
1,881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,{'videourl': 'http://download.ted.com/talks/Da...
2,1757,1,"Привет, я здесь чтобы поговорить с вами о важн...",{'videourl': 'http://download.ted.com/talks/La...
3,1199,1,Я глава отдела развития общественной организац...,{'videourl': 'http://download.ted.com/talks/Ma...
4,1855,6,"Я думаю, мой взгляд на простоту можно отлично ...",{'videourl': 'http://download.ted.com/talks/Al...


Назначаем столбец '@id' индексом фрейма данных, и изменяем его название на более простую строку 'id'.

Также изменяем названия остальных столбцов на ['class','text_original','header'], для нашего удобства.

In [9]:
train.index = train['@id']
train.index.name = 'id'

train = train[['class','content','head']]
train.columns = ['class','text_original','header']
train.head()

,class,text_original,header
id,,,
1848,2,"Я был неописуемо удивлён, узнав о существовани...",{'videourl': 'http://download.ted.com/talks/Ne...
881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,{'videourl': 'http://download.ted.com/talks/Da...
1757,1,"Привет, я здесь чтобы поговорить с вами о важн...",{'videourl': 'http://download.ted.com/talks/La...
1199,1,Я глава отдела развития общественной организац...,{'videourl': 'http://download.ted.com/talks/Ma...
1855,6,"Я думаю, мой взгляд на простоту можно отлично ...",{'videourl': 'http://download.ted.com/talks/Al...


Проверяем сейчас что все ячейки имеют ненулевое значение. За того, что все их являются ненулевыми, нам не нужно удалить строк фрейма данных.

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1640 entries, 1848 to 1171
Data columns (total 3 columns):
class            1640 non-null int64
text_original    1640 non-null object
header           1640 non-null object
dtypes: int64(1), object(2)
memory usage: 51.2+ KB


### Премеры для тестирования

Загружаем файл тестированных премеров аналогично к предыдущему написанному коду, и изменяем названия его столбцов и индекса.

In [11]:
test = pd.read_json('test.json')

test.index = test['@id']
test.index.name = 'id'

test = test[['content','head']]
test.columns = ['text_original','header']
print(test.shape)
test.head()

(410, 2)


,text_original,header
id,,
505,"Когда я делаю свою работу, люди меня ненавидят...",{'videourl': 'http://download.ted.com/talks/Sa...
734,Мой друг-политолог несколько месяцев назад оче...,{'videourl': 'http://download.ted.com/talks/Ad...
1526,"Я - Дэвид Хенсон, и я делаю роботов с характер...",{'videourl': 'http://download.ted.com/talks/Da...
1160,"Вселенная очень велика. Мы живем в галактике, ...",{'videourl': 'http://download.ted.com/talks/Se...
1027,"Если бы ваша жизнь была книгой, а вы — её авто...",{'videourl': 'http://download.ted.com/talks/Am...


# Предварительная обработка текстов

Чтобы увеличить прогнозную точность модели, заставляем все тексты обучающих премеров в процесс предварительной обработки, в течение которой извлекаем все корни слов, представленных в текстах.

### Токенизатор

Начинаем, создавая "токенизатор" библиотеки NLTK, который выявляет все слов от строки образом регулярных выражений, и возвращает список всех найденных терминов.

Также добавляем возвращенный список к новому столбцу фрейма train

In [12]:
tokenizer = nltk.tokenize.RegexpTokenizer('\w+')

train['text_tokenized'] = train['text_original'].apply(tokenizer.tokenize)
train.head()

,class,text_original,header,text_tokenized
id,,,,
1848,2,"Я был неописуемо удивлён, узнав о существовани...",{'videourl': 'http://download.ted.com/talks/Ne...,"[Я, был, неописуемо, удивлён, узнав, о, сущест..."
881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,{'videourl': 'http://download.ted.com/talks/Da...,"[Я, работаю, с, бактериями, Сейчас, я, покажу,..."
1757,1,"Привет, я здесь чтобы поговорить с вами о важн...",{'videourl': 'http://download.ted.com/talks/La...,"[Привет, я, здесь, чтобы, поговорить, с, вами,..."
1199,1,Я глава отдела развития общественной организац...,{'videourl': 'http://download.ted.com/talks/Ma...,"[Я, глава, отдела, развития, общественной, орг..."
1855,6,"Я думаю, мой взгляд на простоту можно отлично ...",{'videourl': 'http://download.ted.com/talks/Al...,"[Я, думаю, мой, взгляд, на, простоту, можно, о..."


### Извлекатель корней

По сокращению объема слов, имеющих одинаковое семантическое значение но определенных по отдельности, создаем и употребляем конструктор RussianStemmer от NLTK на все токенизированные слова.
Этот конструктор извлекает корень данного слова образом его метода .stem(), и возвращает его пользователю. Из-за того, что конструктор по умолчанию не применяет списки, а только строки, нам требуется писать новую функцию, которая итерирует на слова наших списков, и возвращает новый список с извлеченными корнями.

Чтобы выполнить код потребуются несколько минут, 7 или 8.

In [13]:
%%time

from nltk.stem.snowball import RussianStemmer
stemmer = RussianStemmer()

def stemming_from_lists(list_to_stem):
    return [stemmer.stem(x) for x in list_to_stem]

train['text_stemmed'] = train['text_tokenized'].apply(stemming_from_lists) # This takes several minutes

Wall time: 7min 25s


Проверяем, что все случилось как ожиданно

In [14]:
train.head()

,class,text_original,header,text_tokenized,text_stemmed
id,,,,,
1848,2,"Я был неописуемо удивлён, узнав о существовани...",{'videourl': 'http://download.ted.com/talks/Ne...,"[Я, был, неописуемо, удивлён, узнав, о, сущест...","[я, был, неописуем, удивл, узна, о, существова..."
881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,{'videourl': 'http://download.ted.com/talks/Da...,"[Я, работаю, с, бактериями, Сейчас, я, покажу,...","[я, работа, с, бактер, сейчас, я, покаж, вам, ..."
1757,1,"Привет, я здесь чтобы поговорить с вами о важн...",{'videourl': 'http://download.ted.com/talks/La...,"[Привет, я, здесь, чтобы, поговорить, с, вами,...","[привет, я, зде, чтоб, поговор, с, вам, о, важ..."
1199,1,Я глава отдела развития общественной организац...,{'videourl': 'http://download.ted.com/talks/Ma...,"[Я, глава, отдела, развития, общественной, орг...","[я, глав, отдел, развит, обществен, организац,..."
1855,6,"Я думаю, мой взгляд на простоту можно отлично ...",{'videourl': 'http://download.ted.com/talks/Al...,"[Я, думаю, мой, взгляд, на, простоту, можно, о...","[я, дума, мо, взгляд, на, простот, можн, отлич..."


### Конкатенация корней

Из-за того, что употребляемые нами конструкторы машинного обучения не применяют списки строк, но только сами строки, нам нужно 
сейчас инвертировать процесс разделения, используя конкатенацию/сложение строк с корнями слов.

Осуществляем решение этой задачи функцией, описанной ниже.

In [15]:
def attaching_to_a_string(list_of_words):
    only_one_string = ''
    for word in list_of_words:
        only_one_string = only_one_string+word+' '
    return only_one_string

train['text_final'] = train['text_stemmed'].apply(attaching_to_a_string)
train.head()

,class,text_original,header,text_tokenized,text_stemmed,text_final
id,,,,,,
1848,2,"Я был неописуемо удивлён, узнав о существовани...",{'videourl': 'http://download.ted.com/talks/Ne...,"[Я, был, неописуемо, удивлён, узнав, о, сущест...","[я, был, неописуем, удивл, узна, о, существова...",я был неописуем удивл узна о существован орган...
881,2,Я работаю с бактериями. Сейчас я покажу вам сд...,{'videourl': 'http://download.ted.com/talks/Da...,"[Я, работаю, с, бактериями, Сейчас, я, покажу,...","[я, работа, с, бактер, сейчас, я, покаж, вам, ...",я работа с бактер сейчас я покаж вам сдела мно...
1757,1,"Привет, я здесь чтобы поговорить с вами о важн...",{'videourl': 'http://download.ted.com/talks/La...,"[Привет, я, здесь, чтобы, поговорить, с, вами,...","[привет, я, зде, чтоб, поговор, с, вам, о, важ...",привет я зде чтоб поговор с вам о важност похв...
1199,1,Я глава отдела развития общественной организац...,{'videourl': 'http://download.ted.com/talks/Ma...,"[Я, глава, отдела, развития, общественной, орг...","[я, глав, отдел, развит, обществен, организац,...",я глав отдел развит обществен организац робин ...
1855,6,"Я думаю, мой взгляд на простоту можно отлично ...",{'videourl': 'http://download.ted.com/talks/Al...,"[Я, думаю, мой, взгляд, на, простоту, можно, о...","[я, дума, мо, взгляд, на, простот, можн, отлич...",я дума мо взгляд на простот можн отличн проилл...


### Предварительная обработка тестовых премеров

Повторяем целый процесс предварительной обработки текстов для премеров в тестированном фрайме.

Потребуются около двух минуты.

In [16]:
%%time
test['text_tokenized'] = test['text_original'].apply(tokenizer.tokenize)
test['text_stemmed'] = test['text_tokenized'].apply(stemming_from_lists)
test['text_final'] = test['text_stemmed'].apply(attaching_to_a_string)

Wall time: 1min 55s


Проверяем, что все проводилось правильно

In [17]:
test.head()

,text_original,header,text_tokenized,text_stemmed,text_final
id,,,,,
505,"Когда я делаю свою работу, люди меня ненавидят...",{'videourl': 'http://download.ted.com/talks/Sa...,"[Когда, я, делаю, свою, работу, люди, меня, не...","[когд, я, дела, сво, работ, люд, мен, ненавид,...",когд я дела сво работ люд мен ненавид прич чем...
734,Мой друг-политолог несколько месяцев назад оче...,{'videourl': 'http://download.ted.com/talks/Ad...,"[Мой, друг, политолог, несколько, месяцев, наз...","[мо, друг, политолог, нескольк, месяц, назад, ...",мо друг политолог нескольк месяц назад очен то...
1526,"Я - Дэвид Хенсон, и я делаю роботов с характер...",{'videourl': 'http://download.ted.com/talks/Da...,"[Я, Дэвид, Хенсон, и, я, делаю, роботов, с, ха...","[я, дэвид, хенсон, и, я, дела, робот, с, харак...",я дэвид хенсон и я дела робот с характер и эт ...
1160,"Вселенная очень велика. Мы живем в галактике, ...",{'videourl': 'http://download.ted.com/talks/Se...,"[Вселенная, очень, велика, Мы, живем, в, галак...","[вселен, очен, велик, мы, жив, в, галактик, в,...",вселен очен велик мы жив в галактик в галактик...
1027,"Если бы ваша жизнь была книгой, а вы — её авто...",{'videourl': 'http://download.ted.com/talks/Am...,"[Если, бы, ваша, жизнь, была, книгой, а, вы, е...","[есл, бы, ваш, жизн, был, книг, а, вы, е, авто...",есл бы ваш жизн был книг а вы е автор то по ка...


# Создание модели машинного обучения

### Stopwords

Как первый шаг процесса обучения, создаем список чаще всего слов русского языка, чтобы их не включить в векторизатор. Те слова, такие как "а", "но", "и", и т.д., не только не добавляют семантического значения к текстам, но на самом деле уменьшают прогностический потенциал моделей. 

NLKT к счастью предлагает конструктор stopwords, который содержит список всех слов, ненужных нашей модели.

In [18]:
stopwords = nltk.corpus.stopwords.words('russian')

### Входящие и целевые переменные

А теперь указываем обработанные строки от train['text_final'] в входящую переменную нашей модели, и тематические классы от text['class'] в массив с этикетами для схождения с градиентным спуском.

In [19]:
X = train['text_final']
Y = train['class']

### Создание модели и определение его оптимальных гиперпараметров

После создания векторизатора и классификатора, и также пайплайна, содержащего их оба, находим оптимальные параметры векторизатора и классификатора с помощью конструктора GridSearchCV,

Конкурентные параметры, которые входят в процесс пойска сетки, следующие:

#### 1) Для векторизатора:
    
- use_idf: True or False

    рассчитивается ли обратная частота документов (*inverse document frequency* на английском) для повторного взвешивания слов на основе их редкости в текстах.
    Полное описание здесь: http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting
    

- ngram_range: униграммы, униграммы и биграммы, или только биграммы.
    
    Распределяются ли строки текстов на множества единых слов, единых слов и пар, или пар только.
    
#### 2) Для классификатора:

- loss: 'hinge' or 'log'
    
    Целевая функция 'hinge' навязывет модели поведение, аналогичное к линейной SVM; а функция 'log' определяет вероятностный классификатор, аналогичный к модели логистической регрессии.
    
Для расчета требуются около 6 минут на CPU

In [20]:
pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words=stopwords,
                                             strip_accents=None)),
                     ('sgd',SGDClassifier(random_state=42))
                    ])

parameters = {'tfidf__use_idf':(True,False),
              'tfidf__ngram_range':((1,1),(1,2),(2,2)),
              'sgd__loss':('hinge','log')
             }

grid_search = GridSearchCV(pipeline,
                           parameters,
                           n_jobs=-1,                     
                           verbose=1
                          )

t = time.time()
grid_search.fit(X,Y)
print('Computation done in {} seconds'.format(int(time.time()-t)))

print('\nParameters used for GridSearch: ')
pprint(parameters)
print('\nParameters selected as the best fit: ')
pprint(grid_search.best_params_)
print('\n',grid_search.score(X,Y))

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  4.9min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Computation done in 301 seconds

Parameters used for GridSearch: 
{'sgd__loss': ('hinge', 'log'),
 'tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)),
 'tfidf__use_idf': (True, False)}

Parameters selected as the best fit: 
{'sgd__loss': 'log', 'tfidf__ngram_range': (1, 1), 'tfidf__use_idf': True}

 0.9902439024390244


GridSearch решил, что самый оптимальный алгоритм обучения вычисляет логистическую регрессию на единые слова текстов, и взвешивает те слова на основе их обратной частоты в документах.

# Определение тематических классов тестовых премеров

Самый эффективный алгоритм можно использовать для определения классов текстов, наблюдавшихся в течение процесса обучения.

После обучения созданного конструктора, метод .predict() GridSearchCV автоматически взывает метод .predict() самой лучшей обученной модели. Возвращаем таким образом самые вероятные тематические классы тестовых текстов.

In [21]:
predictions = grid_search.predict(test['text_final'])
predictions

array([1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 6, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 1, 4, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 6, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       4, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 6, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 4, 1, 1, 1, 6, 1, 1, 1, 2, 2,
       1, 6, 1, 1, 3, 4, 3, 1, 1, 1, 2, 4, 2, 1, 2, 2, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 3, 2, 1, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 4, 1, 1, 1,
       1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Обратите внимание!

Не понятно почему, но создатели задачи в Kaggle решили пометить тематические классы обучающего файла с 1 по 8, но на самом деле ожидают файл с решениями, содержащий классы тестовых премеров, условное обозначение которых варируется с 0 по 7.
Не ясно почему это сделали. Очень важно так вычесть 1 от массива с прогнозами до создания файла с решениями.

In [22]:
submission = pd.DataFrame({'class':predictions-1}, # IMPORTANT! Keep a -1 here.
                                                   # All classes in the submission_sample are shifted by -1,
                                                   # with comparison to the classes in the training set
                          index=test.index)
submission.index.names = ['id']
submission.head()

,class
id,
505,0
734,0
1526,1
1160,0
1027,0


# Сохранение файла решений и отправление на Kaggle

Те, кто имеют аккаунт в Kaggle, могут отправлять решения задачи на порталь сайта, для автоматического получения оценки.

Ссылка для отправления следующая: https://www.kaggle.com/c/2nd-hw-dl-in-nlp/submit

In [23]:
submission.to_csv('submission.csv')
print('Done')

Done
